`!{sys.executable} -m pip install deltas`

In [17]:
!{sys.executable} -m pip install lxml 

    100% |████████████████████████████████| 5.8MB 298kB/s  eta 0:00:01


In [19]:
import sys
import os
import pandas as pd
import numpy as np
import ast
import datefinder
from bs4 import BeautifulSoup
from collections import Counter
import wikifunctions as wf
import datetime
import lxml

We want to collect the data from a couple of main sources:

```
Revisions of (see afd_talk_revs.py): 
https://en.wikipedia.org/wiki/Wikipedia_talk:WikiProject_Women
https://en.wikipedia.org/wiki/Wikipedia talk:WikiProject Deletion sorting

Cases from:
https://en.wikipedia.org/wiki/Wikipedia:WikiProject_Deletion_sorting/Women/archive
https://en.wikipedia.org/wiki/Wikipedia:WikiProject_Deletion_sorting/Women/archive_2
```



In [2]:
cases = []
for l in ["Wikipedia:WikiProject_Deletion_sorting/Women/archive","Wikipedia:WikiProject_Deletion_sorting/Women/archive_2"]:
    print(l)
    soup = BeautifulSoup(wf.get_page_raw_content(l))
    c = soup.find_all("div", class_="mw-parser-output")[0].find_all('ul')[4].find_all('li')
    cases = cases + c

Wikipedia:WikiProject_Deletion_sorting/Women/archive
Wikipedia:WikiProject_Deletion_sorting/Women/archive_2


In [3]:
len(cases)

13388

In [85]:
cases[345].text.split(' - ')

['Monica Sharma Purohit',
 '(3826)',
 'delete',
 'closed 10:26, 18 March 2020 (UTC)']

In [86]:
c.text.split(' - ')

['Christine Dacera', '(767)', '(unknown)', 'closed (unknown)']

In [95]:
# c is cases[i]
c_array = []
for c in cases:
    #html = c
    url = c.contents[0].get('href')
    name = c.contents[0].get('title')
    if 'nomination)' in name:
        prev_nom = True
    else:
        prev_nom = False
    meta = c.text.split(' - ')
    decision = meta[2]
    try:
        date_utc = [m for m in datefinder.find_dates( meta[3] )][0]
        year = date_utc.year
        month = date_utc.month
    except IndexError:
        date_utc = None
        year = None
        month = None
    c_array.append([name, prev_nom, url, decision, date_utc, year, month])
cases_df = pd.DataFrame(c_array, columns = ['name', 'prev_nom', 'url', 'decision', 'date_utc', 'year','month'])
cases_df['decision'] = cases_df.decision.str.lower().str.strip()
len(cases_df)

cases_df.to_csv('afd_cases_women.tsv',sep='\t',header=True,index=False)

In [130]:
cases_df.loc[cases_df.decision=='merge to charles archibald nicholson'].name.values.tolist()

['Wikipedia:Articles for deletion/Evelyn Louise Nicholson']

In [158]:
wf.get_all_page_revisions("Talk:Sir Charles Nicholson, 2nd Baronet")

,revid,parentid,user,userid,timestamp,size,sha1,contentformat,contentmodel,content,comment,page,date,diff,lag,age
0,111805397,0,Mcginnly,1119340,2007-03-01 11:28:08+00:00,42,dc3597e85a1b67637218134ada5dcdd5ba0fcc39,text/x-wiki,wikitext,{{Architecture|class=stub|importance=mid}},[[WP:AES|←]]Created page with '{{Architecture|...,"Talk:Sir Charles Nicholson, 2nd Baronet",2007-03-01,NaN,NaN,0.000000
1,122262839,111805397,Kingbotk,1390043,2007-04-12 17:12:02+00:00,103,ab74ba1e04215a04c5a8bc3f9dcb4db0851b511a,text/x-wiki,wikitext,{{WPBiography\n|living=no\n|class=Stub\n|prior...,Bot ([[User:Kingbotk#FAQ|FAQ]]) People stubs/2...,"Talk:Sir Charles Nicholson, 2nd Baronet",2007-04-12,61.0,3649434.0,42.238819
2,167579774,122262839,Tryde,950157,2007-10-28 06:24:00+00:00,103,ab74ba1e04215a04c5a8bc3f9dcb4db0851b511a,text/x-wiki,wikitext,{{WPBiography\n|living=no\n|class=Stub\n|prior...,moved [[Talk:Charles Archibald Nicholson]] to ...,"Talk:Sir Charles Nicholson, 2nd Baronet",2007-10-28,0.0,17154718.0,240.788796
3,276118217,167579774,Yobot,7328338,2009-03-09 21:35:18+00:00,128,0e50db305922ee01c09d7a671878d38ca9032568,text/x-wiki,wikitext,{{WPBiography\n|living=no\n|class=Stub\n|prior...,Tagging([[User:Kingbotk/P|Plugin++]]) baronets...,"Talk:Sir Charles Nicholson, 2nd Baronet",2009-03-09,25.0,43081878.0,739.421644
4,338073284,276118217,ListasBot,9201969,2010-01-15 22:01:53+00:00,117,884bb814571c680bec4028756cf238adee85cf27,text/x-wiki,wikitext,{{WPBiography\n|living=no\n|class=Stub\n|auto=...,Applied fixes to WPBiography template. [[User...,"Talk:Sir Charles Nicholson, 2nd Baronet",2010-01-15,-11.0,26958395.0,1051.440104
5,412428477,338073284,EricSerge,3192076,2011-02-06 22:24:25+00:00,160,df00d210760f4044d9619d1c7793802d33cb4d29,text/x-wiki,wikitext,{{WPBiography\n|living=no\n|class=Stub\n|auto=...,,"Talk:Sir Charles Nicholson, 2nd Baronet",2011-02-06,43.0,33438152.0,1438.455752
6,502799043,412428477,Meno25,1255878,2012-07-17 14:14:46+00:00,151,ac6610cf425743baad06e5a6d2dcff710501d39d,text/x-wiki,wikitext,{{WPBiography\n|living=no\n|class=Stub\n\n|bar...,Reviewed rating using [[Project:AWB|AWB]] (8097),"Talk:Sir Charles Nicholson, 2nd Baronet",2012-07-17,-9.0,45503421.0,1965.115718
7,616738819,502799043,Yobot,7328338,2014-07-13 03:46:12+00:00,172,5b135e2bd0cec95c6649ede1947eadc7a40d282a,text/x-wiki,wikitext,{{WikiProject Biography|living=no\n|class=Stub...,Tagging for WikiProject Architecture - step 1 ...,"Talk:Sir Charles Nicholson, 2nd Baronet",2014-07-13,21.0,62688686.0,2690.679213
8,741117805,616738819,PKT,1382933,2016-09-25 14:00:23+00:00,174,703b545218034264f817b8d89158a8008cf5b319,text/x-wiki,wikitext,{{WikiProject Biography|living=no\n|class=Star...,class=start,"Talk:Sir Charles Nicholson, 2nd Baronet",2016-09-25,2.0,69588851.0,3496.105729


In [155]:
cases_df

,name,prev_nom,url,decision,date_utc,year,month
0,Wikipedia:Articles for deletion/Angelica Barcelo,False,/wiki/Wikipedia:Articles_for_deletion/Angelica...,delete,2021-01-17 16:24:00,2021.0,1.0
1,Wikipedia:Articles for deletion/Philippines at...,False,/wiki/Wikipedia:Articles_for_deletion/Philippi...,delete,2020-05-05 08:25:00,2020.0,5.0
2,Wikipedia:Articles for deletion/Mary Cruz,False,/wiki/Wikipedia:Articles_for_deletion/Mary_Cruz,delete,2020-05-05 08:35:00,2020.0,5.0
3,Wikipedia:Articles for deletion/Paola Morra,False,/wiki/Wikipedia:Articles_for_deletion/Paola_Morra,delete,2020-05-05 08:27:00,2020.0,5.0
4,Wikipedia:Articles for deletion/Lee Barnes (cook),False,/wiki/Wikipedia:Articles_for_deletion/Lee_Barn...,withdrawn,2020-05-04 18:10:00,2020.0,5.0
...,...,...,...,...,...,...,...
13383,Wikipedia:Articles for deletion/Princess Nanas...,False,/wiki/Wikipedia:Articles_for_deletion/Princess...,redirect,2020-05-05 22:16:00,2020.0,5.0
13384,Wikipedia:Articles for deletion/Playlist: The ...,True,/wiki/Wikipedia:Articles_for_deletion/Playlist...,redirect,2020-05-05 23:09:00,2020.0,5.0
13385,Wikipedia:Articles for deletion/Charlotte Lawr...,True,/wiki/Wikipedia:Articles_for_deletion/Charlott...,keep,2020-05-06 05:12:00,2020.0,5.0
13386,Wikipedia:Articles for deletion/Narmina Afandi...,False,/wiki/Wikipedia:Articles_for_deletion/Narmina_...,delete,2020-05-06 07:22:00,2020.0,5.0


'Sir Charles Nicholson, 2nd Baronet'

In [ ]:

#<span class="mw-page-title-main">Sir Charles Nicholson, 2nd Baronet</span>

'<div class="mw-parser-output"><div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">British architect (1867–1949)</div>\n<p class="mw-empty-elt">\n</p>\n<style data-mw-deduplicate="TemplateStyles:r1066479718">.mw-parser-output .infobox-subbox{padding:0;border:none;margin:-3px;width:auto;min-width:100%;font-size:100%;clear:none;float:none;background-color:transparent}.mw-parser-output .infobox-3cols-child{margin:auto}.mw-parser-output .infobox .navbar{font-size:100%}body.skin-minerva .mw-parser-output .infobox-header,body.skin-minerva .mw-parser-output .infobox-subheader,body.skin-minerva .mw-parser-output .infobox-above,body.skin-minerva .mw-parser-output .infobox-title,body.skin-minerva .mw-parser-output .infobox-image,body.skin-minerva .mw-parser-output .infobox-full-data,body.skin-minerva .mw-parser-output .infobox-below{text-align:center}</style><table class="infobox biography vcard"><tbody><tr><th colspan="2" class="infobox-above" style="font-siz

In [122]:
str([c.lower() for c in wf.get_category_memberships("Talk:Lee Barnes (cook)") if ('importance' in c.lower())])

"['category:low-importance food and drink articles', 'category:low-importance louisiana articles', 'category:low-importance new orleans articles', 'category:low-importance united states articles', 'category:start-class united states articles of low-importance']"

In [105]:
decision_types_counts = Counter(cases_df.decision.values.tolist())

In [125]:
decision_types_counts

Counter({'delete': 7198,
         'withdrawn': 33,
         'speedy delete': 142,
         'keep': 3011,
         'no consensus': 854,
         'procedural close': 14,
         'merge': 136,
         'redirect': 546,
         'speedy keep': 293,
         'merge to charles archibald nicholson': 1,
         'speedy deleted per g5': 1,
         'speedy delete and protected redirect': 1,
         'soft delete': 665,
         'no consensus to delete': 16,
         'deleted': 8,
         'deleted as a7': 1,
         'withdrawn by nominator': 12,
         'delete and redirect': 67,
         'deleted as g11': 2,
         'nomination withdrawn': 20,
         'speedy deleted': 12,
         'withdrawn by nominator as substantially improved': 1,
         'move': 1,
         'draftify': 59,
         'wp:snow keep': 9,
         'merge to the fold (brand)': 1,
         'keep but discuss a move': 1,
         'weak keep': 4,
         '(unknown)': 25,
         'speedily deleted by dgg': 1,
         'pro